In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from google.colab import auth
from google.colab import drive
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import math as math

In [0]:
drive.mount('/content/gdrive') #mount the googledrive
#!ls "/content/gdrive/My Drive/DA_Coursework/3. Data" #list the content

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
#2READ DATASET FROM FILE
s_boroughs = pd.read_csv("/content/gdrive/My Drive/DA_Coursework/3. Data/londonBoroughs.csv")
#s_boroughs = pd.read_csv("/content/gdrive/My Drive/DA_Coursework/3. Data/londonBoroughsWO_Others.csv")
#s_boroughs = pd.read_csv("/content/gdrive/My Drive/DA_Coursework/3. Data/londonBoroughsWO_Others.csv")
#s_boroughs = pd.read_csv("/content/gdrive/My Drive/DA_Coursework/3. Data/londonBoroughs_realprice.csv")

In [0]:
#3UNIQUE VALUES IN 'district' COLUMN
set(s_boroughs['district'])

{'CITY OF WESTMINSTER', 'CROYDON', 'LAMBETH'}

In [0]:
#4Convert Date to numerical value for the three datasets:
s_boroughs['dateOfTransfer'] = pd.to_datetime(s_boroughs['dateOfTransfer'])
s_boroughs['dateOfTransfer'] = s_boroughs['dateOfTransfer'].map(dt.datetime.toordinal)

In [0]:
#5Create the dummy values for categorical values (Property Type, Old/New, Duration, PPD)
s_boroughs = pd.concat([s_boroughs,pd.get_dummies(s_boroughs['propertyType'], prefix='prop_type')],axis=1)
s_boroughs = pd.concat([s_boroughs,pd.get_dummies(s_boroughs['oldNew'], prefix='oldNew')],axis=1)
s_boroughs = pd.concat([s_boroughs,pd.get_dummies(s_boroughs['duration'], prefix='duration')],axis=1)

In [0]:
#6Divide the dataset in three boroughs:
df_lambeth = s_boroughs.loc[s_boroughs['district']=='LAMBETH'] 
df_croydon = s_boroughs.loc[s_boroughs['district']=='CROYDON']
df_westminster = s_boroughs.loc[s_boroughs['district']=='CITY OF WESTMINSTER']

In [0]:
#7Drop the features that will not be considered:
df_lambeth.drop(columns=['Unnamed: 0','townCity','district','county','propertyType','oldNew','duration'],inplace=True)
df_croydon.drop(columns=['Unnamed: 0','townCity','district','county','propertyType','oldNew','duration'],inplace=True)
df_westminster.drop(columns=['Unnamed: 0','townCity','district','county','propertyType','oldNew','duration'],inplace=True)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
#CALCULATE THE MEAN PER DAY
df_lambeth = df_lambeth.groupby('dateOfTransfer').mean()
df_croydon = df_croydon.groupby('dateOfTransfer').mean()
df_westminster = df_westminster.groupby('dateOfTransfer').mean()

In [0]:
#8EXTRACT THE FEATURES OF EACH BOROUGH FOR X AND Y VARIABLES
xfeat = ['prop_type_D', 'prop_type_F','prop_type_S', 'prop_type_T','oldNew_N', 'oldNew_Y', 'duration_F','duration_L', 'duration_U']
yfeat = ['price']
#X AND Y FOR WESTMINSTER
Xw = df_westminster[xfeat]
Yw = df_westminster[yfeat]
#X AND Y FOR CROYDON
Xc = df_croydon[xfeat]
Yc = df_croydon[yfeat]
#X AND Y FOR LAMBETH
Xl = df_lambeth[xfeat]
Yl = df_lambeth[yfeat]

#SPLIT THE TRAIN AND TEST DATASET
Xw_train, Xw_test, Yw_train, Yw_test = train_test_split(Xw, Yw, test_size=0.2,random_state = 42)
Xc_train, Xc_test, Yc_train, Yc_test = train_test_split(Xc, Yc, test_size=0.2,random_state = 42)
Xl_train, Xl_test, Yl_train, Yl_test = train_test_split(Xl, Yl, test_size=0.2,random_state = 42)


#Linear regression model

In [0]:
#10BUILDING THE REGRESSION MODEL FOR WESTMINSTER
#WESTMINSTER
modelw = LinearRegression()
# Lambeth
modell = LinearRegression()
# Croydon
modelc = LinearRegression()

#TRAINING THE MODEL FOR EACH BOROUGH
modelw.fit(Xw_train, Yw_train)
modell.fit(Xl_train, Yl_train)
modelc.fit(Xc_train, Yc_train)

# MODEL PREDICTIONS 
Yw_predict = modelw.predict(Xw_test)
Yl_predict = modell.predict(Xl_test)
Yc_predict = modelc.predict(Xc_test)

#CONVERT TO DATAFRAME
Yw_predict = pd.DataFrame(Yw_predict)
Yl_predict = pd.DataFrame(Yl_predict)
Yc_predict = pd.DataFrame(Yc_predict)


# **Polynomial Regression**

In [0]:
#POLYNOMIAL REGRESSION: Building the model:Croydon

#Transform X_train and X_test into polynomials order 2
polyC = PolynomialFeatures(degree=2, interaction_only=True)
Xc_train_Poly = polyC.fit_transform(Xc_train)
Xc_test_Poly = polyC.fit_transform(Xc_test)


#Build the model
polyC_model = LinearRegression()
polyC_model.fit(Xc_train_Poly, Yc_train)

#Predict Y Values
yc_PolyPredict = polyC_model.predict(Xc_test_Poly)


#X_test.columns
yc_PolyPredict = pd.DataFrame(yc_PolyPredict)
yc_PolyPredict.head(5)

,0
0,164352.0
1,131072.0
2,187904.0
3,187904.0
4,178176.0


In [0]:
#POLYNOMIAL REGRESSION: Building the model: Westminster


#Transform X_train and X_test into polynomials order 2
polyW = PolynomialFeatures(degree=2, interaction_only=True)
Xw_train_Poly = polyW.fit_transform(Xw_train)
Xw_test_Poly = polyW.fit_transform(Xw_test)


#Build the model
polyW_model = LinearRegression()
polyW_model.fit(Xw_train_Poly, Yw_train)

#Predict Y Values
yw_PolyPredict = polyW_model.predict(Xw_test_Poly)


#X_test.columns
yw_PolyPredict = pd.DataFrame(yw_PolyPredict)
yw_PolyPredict.head(5)

,0
0,423936.0
1,724992.0
2,770048.0
3,299008.0
4,851968.0


In [0]:
#POLYNOMIAL REGRESSION: Building the model: Lambeth


#Transform X_train and X_test into polynomials order 2
polyL = PolynomialFeatures(degree=2, interaction_only=True)
Xl_train_Poly = polyL.fit_transform(Xl_train)
Xl_test_Poly = polyL.fit_transform(Xl_test)


#Build the model
polyL_model = LinearRegression()
polyL_model.fit(Xl_train_Poly, Yl_train)

#Predict Y Values
yl_PolyPredict = polyL_model.predict(Xl_test_Poly)


#X_test.columns
yl_PolyPredict = pd.DataFrame(yl_PolyPredict)
yl_PolyPredict.head(5)

,0
0,299520.0
1,305152.0
2,577536.0
3,296448.0
4,300544.0


##Score

In [0]:
#OBTAIN THE SCORE MODELS
modelw.score(Xw_test, Yw_test)

0.392072363498611

# MSE calculation

In [0]:
#Root MSE for each linear regression
#Westminster
MSEw = math.sqrt(mean_squared_error(Yw_test,Yw_predict))
print('MSE Westminster:',MSEw)
#Croydon
MSEc = math.sqrt(mean_squared_error(Yc_test,Yc_predict))
print('MSE Croydon:',MSEc)
#Lambeth
MSEl = math.sqrt(mean_squared_error(Yl_test,Yl_predict))
print('MSE Lambeth:',MSEl)

MSE Westminster: 1359742.1917314979
MSE Croydon: 114397.6302481525
MSE Lambeth: 212529.62145090871


In [0]:
#Root MSE for each polynomial regression
#Westminster
P_MSEw = math.sqrt(mean_squared_error(Yw_test, yw_PolyPredict))
print('P_MSE Westminster:',P_MSEw)
#Croydon
P_MSEc = math.sqrt(mean_squared_error(Yc_test, yc_PolyPredict))
print('P_MSE Croydon:',P_MSEc)
#Lambeth
P_MSEl = math.sqrt(mean_squared_error(Yl_test, yl_PolyPredict))
print('P_MSE Lambeth:',P_MSEl)

P_MSE Westminster: 1640556.1985620223
P_MSE Croydon: 3576066.6873270255
P_MSE Lambeth: 229921480224270.2


##MSE values for complete dataset:


MSE Westminster: 15228521970000.19

MSE Croydon: 324078593900.4582

MSE Lambeth: 433597856486.83826

##MSE values for  dataset without others:
MSE Westminster: 1220781555722.7695

MSE Croydon: 8622581812.974504

MSE Lambeth: 46780201580.45319

In [0]:
MAEw = mean_absolute_error(Yw_test,Yw_predict)
print('MAE Westminster:',MSEw)
#Croydon
MAEc = mean_absolute_error(Yc_test,Yc_predict)
print('MAE Croydon:',MSEc)
#Lambeth
MAEl = mean_absolute_error(Yl_test,Yl_predict)
print('MAE Lambeth:',MSEl)

##MAE values for  dataset without others:
MAE Westminster: 452899.3690011841

MAE Croydon: 53972.11482989346

MAE Lambeth: 120646.35975298125

In [0]:
#Compare actual price vs result of the regression
comparison = pd.concat([Yw_predict,Yw_test],axis=1,ignore_index = True)
Yw_predict


#Using cross validation

In [0]:
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import metrics

In [0]:
# Perform 6-fold cross validation
scores = cross_val_score(modelw,Xw_train, Yw_train, cv=10)
print("Cross-validated scores:", scores)

In [0]:
predictions = cross_val_predict(modelw, Xw_test, Yw_test, cv=8)
plt.scatter(Yw_test, predictions)

In [0]:
accuracy = metrics.r2_score(Yw_test, predictions)
print("Cross-Predicted Accuracy:", accuracy)